# Set up environment

**Import libraries**

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import time

Relevant functions from github file

In [ ]:
def image_pooling(image, new_width, new_height, cvt_color):
    img =  cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
    if cvt_color:
        return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    else:
      return img

def computeRollingVariance(square_sum, sum, num_elements):
    return (square_sum/num_elements-(sum/num_elements)**2)

Functions for datasets

In [ ]:
def getVariances(video_path, frame_width=128, frame_height=128, target_frame_width=28, target_frame_height=28):
    """
    video_path is the file path to the input video
    Uses functions in the github

    Returns:
    (diff_variances, input_variances)
    """
    num_frame = 5
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(5)
    total_frames = int(cap.get(7))-num_frame+1
    print('Video loaded with', total_frames,'frames')

    diff_variances = np.zeros((total_frames,target_frame_height,target_frame_width),dtype=np.float64)
    input_variances = np.zeros((total_frames,target_frame_height,target_frame_width),dtype=np.float64)
    #Constants below
    input_frames = np.zeros((num_frame, frame_height, frame_width), dtype=np.float32)
    derivative1 = np.zeros((2, frame_height, frame_width), dtype=np.float32)
    cur_derivative2_corrected = np.zeros((frame_height, frame_width), dtype=np.float32)
    differences = np.zeros((num_frame, frame_height, frame_width), dtype=np.float32)
    start_frame = 0
    rotation_frequency = 1
    frequency_const = rotation_frequency*2*np.pi
    difference_sum = np.sum(differences,axis=0)
    difference_square_sum = np.sum(np.square(differences),axis=0)
    input_sum = np.sum(input_frames,axis=0)
    input_square_sum = np.sum(np.square(input_frames),axis=0)
    frame_num=0
    start_time = time.time()
    delta_time = 1
    started = False
    # Next part loops through each frame in video
    while True:
        ret, frame = cap.read()
        if ret:
            input_sum -= input_frames[start_frame]
            input_square_sum -= np.square(input_frames[start_frame])
            difference_sum -= differences[start_frame-1]
            difference_square_sum -= np.square(differences[start_frame-1])
            # read in image
            input_frames[start_frame] = image_pooling(
                frame, frame_width, frame_height, True)/255

            # compute first derivative
            derivative1[start_frame % 2] = (input_frames[start_frame]-input_frames[(start_frame-1)])/delta_time

            # compute second derivative and correct its coefficient
            cur_derivative2_corrected = (derivative1[start_frame % 2]-derivative1[(start_frame-1) % 2])/delta_time
            cur_derivative2_corrected /= frequency_const**2

            # compute difference between image and its second derivative. It's actually a +
            # because of the negative sign from differentiation
            differences[start_frame-1] = cur_derivative2_corrected + \
                input_frames[(start_frame-1)]

            # add in new variance of the newly read in image and newly computed difference
            input_sum += input_frames[start_frame]
            input_square_sum += input_frames[start_frame]**2
            difference_sum += differences[start_frame-1]
            difference_square_sum += differences[start_frame-1]**2

            # recompute variances
            input_variance = computeRollingVariance(input_square_sum,input_sum,num_frame)
            variances = computeRollingVariance(difference_square_sum,difference_sum,num_frame)
            # note this is only an estimation of variance, not the actual variance, which may be difficult
            # to evaluate on a rolling basis

            # scale down variance to ensure connectiveness
            diff_variances[frame_num] = image_pooling(
                variances, target_frame_width, target_frame_height, False)
            input_variances[frame_num] = image_pooling(
                input_variance, target_frame_width, target_frame_height, False)
            start_frame = (start_frame+1) % num_frame
        else:
            break
        if start_frame == 0 :
            started = True
        if started :
            frame_num += 1
        delta_time = time.time()-start_time
        start_time = time.time()
    cap.release()
    return diff_variances, input_variances

In [ ]:
def variancesToDataset(diff_variance,input_variance,window_location, batch_size=16):
    """
    diff_variance and input_variance are three dimensional numpy arrays, with third dimension being frame number
    Before inputting into this function, stack every frame from all images to diff_variance and input_variance
    window_location is binary for whether or not there is a window at a pixel in a certain frame

    Returns:
    Dataset
    """
    features = np.transpose(np.array((diff_variance.flatten(),input_variance.flatten())))
    data = (features,window_location.flatten())
    ds = tf.data.Dataset.from_tensor_slices(data)
    return ds.batch(batch_size).prefetch(1)

In [ ]:
def get2D(diff_variances, input_variances):
    """
    diff_variance and input_variance are three dimensional numpy arrays, with third dimension being frame number
    Before inputting into this function, stack every frame from all images to diff_variance and input_variance
    
    Returns:
    2d numpy array with [variance, variance] as each row
    """
    return np.transpose(np.array((diff_variances.flatten(),input_variances.flatten())))

# Preprocess Data

In [ ]:
diff, input = getVariances('Clock_Face_2Videvo.mov')

Video loaded with -4 frames


ValueError: ignored

In [ ]:
input

array([[[2.31585000e-06, 1.44413207e-06, 2.30170554e-06, ...,
         9.33068804e-07, 1.21950870e-06, 1.60699710e-06],
        [5.89294359e-07, 1.29319960e-06, 1.63790537e-06, ...,
         1.49413245e-06, 1.55856833e-06, 1.37591269e-06],
        [1.01700425e-06, 1.66683458e-06, 9.30274837e-07, ...,
         2.32581806e-06, 1.60248601e-06, 2.52178870e-06],
        ...,
        [2.36227061e-06, 1.43678335e-06, 2.25746226e-06, ...,
         7.90052582e-07, 2.64844857e-07, 9.46107320e-07],
        [1.10489782e-06, 1.36080598e-06, 2.73459773e-06, ...,
         2.89292075e-07, 1.11537520e-06, 1.71031570e-06],
        [2.08011625e-06, 1.66519749e-06, 2.20309630e-06, ...,
         5.65545633e-07, 5.25440555e-07, 1.80193456e-06]],

       [[1.55158341e-06, 1.40897464e-06, 1.91468280e-06, ...,
         1.02986814e-06, 1.19727338e-06, 1.60129275e-06],
        [4.93018888e-07, 1.18173193e-06, 1.51229324e-06, ...,
         1.22172059e-06, 1.72748696e-06, 1.26875238e-06],
        [7.78236426e-07, 

In [ ]:
a = np.ones((180,28,28)) # 6 seconds at 30fps, 28x28 pixels
a[0][0][2] = 5
b = 2*np.ones((80,28,128))
c = np.zeros((80,28,28)).flatten()
features = get2D(diff,input)
features

array([[7.93635693e+03, 1.50596619e-01],
       [8.46031250e+03, 9.23194885e-02],
       [8.52368848e+03, 1.49574280e-01],
       ...,
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00]])

In [ ]:
ds = variancesToDataset(diff, input, np.zeros((219,28,28)))
ds

<PrefetchDataset shapes: ((None, 2), (None,)), types: (tf.float64, tf.float64)>

# Create model

In [ ]:
EPOCHS = 2
BATCH_SIZE = 512

In [ ]:
model = tf.keras.models.Sequential([
    #tf.keras.layers.Dense(2, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(1, activation='sigmoid',input_shape=(2,))
])

model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(features,c,batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2)

NameError: ignored

In [ ]:
model.layers[0].get_weights()